In [2]:
import numpy as np

from QC_DFT.NeuralNetwork import QCDFTModel
from QC_DFT.Circuit import QCDFT_Circuit, NN_QCDFT_Circuit
from QC_DFT.Utils import Gates
import pennylane as qml

import random
import copy

### Random circuit generator

In [15]:
def generate_random_gates(n_qubits, n_gates):
 
    gates = ["X", "Y", "Z", "H", "CX", "CX", "CX", "CX", "CX", "CX", "CX", "CX", "RX", "RY", "RZ"]
    qubits = list(range(n_qubits))

    res = []
    
    for i in range(n_gates):
        gate = random.choice(gates)
        qubit = random.choice(qubits)
        param = None
        if gate == "CX":
            qubit2 = None
            while qubit2 == None or qubit2 == qubit:
                qubit2 = random.choice(qubits)
            qubit = (qubit, qubit2)
        if gate == "RX" or gate == "RY" or gate == "RZ":
            param = random.uniform(0, 2 * np.pi)
        res += [(gate, qubit, param)]
    
    return res

### Exact simulation

In [16]:
def apply_gate_exact(state, gate):
    n_qubits = int(np.log2(state.shape[0]))
    gate, qubit, params = gate

    unitary = None
    if gate == "X":
        unitary = Gates.X()
    if gate == "Y":
        unitary = Gates.Y()
    if gate == "Z":
        unitary = Gates.Z()
    if gate == "H":
        unitary = Gates.H()
    if gate == "I":
        unitary = Gates.I()
    if gate == "RX":
        unitary = Gates.RX(params)
    if gate == "RY":
        unitary = Gates.RY(params)
    if gate == "RZ":
        unitary = Gates.RZ(params)

    if unitary is not None:
        for _ in range(qubit):
            unitary = np.kron(Gates.I(), unitary)
        for _ in range(qubit + 1, n_qubits):
            unitary = np.kron(unitary, Gates.I())

    if gate == "CX":
        unitary = np.zeros((2 ** n_qubits, 2 ** n_qubits), dtype=complex)
        for i in range(2 ** n_qubits):
            if i & (2 ** qubit[0]) != 0:
                unitary[i][i ^ (2 ** qubit[1])] = 1
            else:
                unitary[i][i] = 1

    return unitary @ state @ np.transpose(np.conjugate(unitary))

### Bernardi and 1-RDM circuit

In [17]:
def apply_gate_circuit(circuit, gate):
    gate, qubit, params = gate
    if gate == "X":
        circuit.x(qubit)
    if gate == "Y":
        circuit.y(qubit)
    if gate == "Z":
        circuit.z(qubit)
    if gate == "H":
        circuit.h(qubit)
    if gate == "I":
        circuit.i(qubit)
    if gate == "RX":
        circuit.rx(params, qubit)
    if gate == "RY":
        circuit.ry(params, qubit)
    if gate == "RZ":
        circuit.rz(params, qubit)
    if gate == "CX":
        circuit.cx(*qubit)

    return circuit

### SQP and fidelity calculation

In [ ]:
def get_sqp_exact(state):
    n_qubits = int(np.log2(state.shape[0]))

    sqps = [0.0 for _ in range(n_qubits)]

    for i in range(2 ** n_qubits):
        for j in range(n_qubits):
            if i & (2 ** j) != 0:
                sqps[j] += np.real(state[i][i]) 

    return np.array(sqps, dtype=float)


def get_fidelity(state, rdms):
    n_qubits = int(np.log2(state.shape[0]))
    trace_list = list(range(n_qubits))
    fidelity = []
    for i in range(n_qubits):
        trace_list_ = copy.deepcopy(trace_list)
        trace_list_.remove(i)
        fidelity += [qml.math.fidelity(qml.math.partial_trace(state, trace_list_), rdms[i])]
    return np.array(fidelity)

### 1-RDM neural network model

In [19]:
nn_models = QCDFTModel.load("Models/default")

print(nn_models.control_nn)
print(nn_models.target_nn)

QCDFTPytorchModel(
  (flatten_): Flatten(start_dim=1, end_dim=-1)
  (linears_): ModuleList(
    (0): Linear(in_features=16, out_features=5, bias=True)
    (1): Linear(in_features=5, out_features=64, bias=True)
    (2): Sigmoid()
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): Sigmoid()
    (5): Linear(in_features=64, out_features=128, bias=True)
    (6): Sigmoid()
    (7): Linear(in_features=128, out_features=256, bias=True)
    (8): Sigmoid()
    (9): Linear(in_features=256, out_features=512, bias=True)
    (10): Sigmoid()
    (11): Linear(in_features=512, out_features=1024, bias=True)
    (12): Sigmoid()
    (13): Linear(in_features=1024, out_features=16, bias=True)
  )
)
QCDFTPytorchModel(
  (flatten_): Flatten(start_dim=1, end_dim=-1)
  (linears_): ModuleList(
    (0): Linear(in_features=16, out_features=5, bias=True)
    (1): Linear(in_features=5, out_features=64, bias=True)
    (2): Sigmoid()
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4)

### Random circuit parameters

In [20]:
iteration_count = 300
n_qubits = 5
n_gates = 100

# maximum bond dimension list
bond_dim_list = [2, 3, 4, 5]

# contraction method (auto-mps, swap+split, or nonlocal)
contraction = "auto-mps"

# Output folder
folder = "Results"

### Generate arrays

In [21]:
for i in range(len(bond_dim_list)):
    print("\n maximum bond dimension = ", bond_dim_list[i], "\n")

    exact_sqp_mean = []

    bernardi_sqp_mean = []
    bernardi_sqp_error = []
    bernardi_fidelity_mean = []

    onerdm_sqp_mean = []
    onerdm_sqp_error = []
    onerdm_fidelity_mean = []

    mps_sqp_mean = []
    mps_sqp_error = []
    mps_fidelity_mean = []

    # Define the keyword arguments for the MPS method
    kwargs_mps = {
        # Maximum bond dimension of the MPS
        "max_bond_dim": bond_dim_list[i],
        # Cutoff parameter for the singular value decomposition
        "cutoff": np.finfo(np.complex128).eps,
        # Contraction strategy to apply gates
        "contract": contraction,
    }

    # Instantiate the device with the MPS method and the specified kwargs
    dev = qml.device("default.tensor", method="mps", **kwargs_mps, wires=n_qubits)
    
    for iteration in range(iteration_count):

        # generate a random circuit
        gates_list = generate_random_gates(n_qubits, n_gates)

        print(f"--------- iteration = {iteration + 1} ---------")

        # initialize the circuit
        exact_state = np.zeros((2 ** n_qubits, 2 ** n_qubits), dtype=complex)
        exact_state[0][0] = 1
        bernardi_circuit = QCDFT_Circuit(n_qubits)
        onerdm_circuit = NN_QCDFT_Circuit(n_qubits, nn_models)

        exact_sqp_mean_inner = []

        bernardi_sqp_mean_inner = []
        bernardi_sqp_error_inner = []
        bernardi_fidelity_mean_inner = []

        onerdm_sqp_mean_inner = []
        onerdm_sqp_error_inner = []
        onerdm_fidelity_mean_inner = []
    
        mps_sqp_mean_inner = []
        mps_sqp_error_inner = []
        mps_fidelity_mean_inner = []

        # mps simulation
        for j in range(n_gates):
            if (j == 0):
                @qml.qnode(dev)
                def circuit():
                    gate, qubit_ind, theta = gates_list[j]

                    if gate == "X":
                        qml.X(wires=qubit_ind)
                    if gate == "Y":
                        qml.Y(wires=qubit_ind)
                    if gate == "Z":
                        qml.Z(wires=qubit_ind)
                    if gate == "H":
                        qml.Hadamard(wires=qubit_ind)
                    if gate == "I":
                        qml.I(wires=qubit_ind)
                    if gate == "RX":
                        qml.RX(theta, wires=qubit_ind)
                    if gate == "RY":
                        qml.RY(theta, wires=qubit_ind)
                    if gate == "RZ":
                        qml.RZ(theta, wires=qubit_ind)
                    if gate == "CX":
                        qml.CNOT(wires=[qubit_ind[0], qubit_ind[1]])
                    
                    return qml.state()
                
                state_intermediate = circuit()

            elif (j != 0):
                @qml.qnode(dev)
                def circuit():
                    gate, qubit_ind, theta = gates_list[j]
                    qml.StatePrep(state_intermediate, wires=range(n_qubits), normalize=True)

                    if gate == "X":
                        qml.X(wires=qubit_ind)
                    if gate == "Y":
                        qml.Y(wires=qubit_ind)
                    if gate == "Z":
                        qml.Z(wires=qubit_ind)
                    if gate == "H":
                        qml.Hadamard(wires=qubit_ind)
                    if gate == "I":
                        qml.I(wires=qubit_ind)
                    if gate == "RX":
                        qml.RX(theta, wires=qubit_ind)
                    if gate == "RY":
                        qml.RY(theta, wires=qubit_ind)
                    if gate == "RZ":
                        qml.RZ(theta, wires=qubit_ind)
                    if gate == "CX":
                        qml.CNOT(wires=[qubit_ind[0], qubit_ind[1]])
                    
                    return qml.state()
                
                state_intermediate = circuit()

            state_vector_mps = state_intermediate.reshape(-1, 1)
            rho_mps = state_vector_mps @ np.transpose(np.conjugate(state_vector_mps))

            # exact simulation
            exact_state = apply_gate_exact(exact_state, gates_list[j])
            exact_sqp = get_sqp_exact(exact_state)
            exact_sqp_mean_inner += [np.mean(exact_sqp)]

            # Bernardi and 1-RDM simulation
            bernardi_circuit = apply_gate_circuit(bernardi_circuit, gates_list[j])
            onerdm_circuit = apply_gate_circuit(onerdm_circuit, gates_list[j])

            bernardi_state = bernardi_circuit.evolve(1)
            onerdm_state = onerdm_circuit.evolve(1)

            bernardi_sqp = np.real(bernardi_state[:,1,1])
            onerdm_sqp = np.real(onerdm_state[:,1,1])

            exact_sqp_mean_inner += [np.mean(exact_sqp)]
            bernardi_sqp_mean_inner += [np.mean(bernardi_sqp)]
            onerdm_sqp_mean_inner += [np.mean(onerdm_sqp)]

            bernardi_sqp_error_inner += [np.sqrt(np.mean(np.square(exact_sqp - bernardi_sqp)))]
            onerdm_sqp_error_inner += [np.sqrt(np.mean(np.square(exact_sqp - onerdm_sqp)))]

            bernardi_fidelity_mean_inner += [np.mean(get_fidelity(exact_state, bernardi_state))]
            onerdm_fidelity_mean_inner += [np.mean(get_fidelity(exact_state, onerdm_state))]

            # mps sqp and fidelity
            arr = np.arange(0, n_qubits)
            mps_sqp = []
            mps_rdm = []
            for k in range(n_qubits):
                arrj = arr[arr != k]
                rdmk = qml.math.partial_trace(rho_mps, indices=arrj)
                mps_sqp.append(np.real(rdmk[1, 1]))
                mps_rdm.append(rdmk)
            mps_sqp = np.array(mps_sqp)
            mps_rdm = np.array(mps_rdm)            

            mps_sqp_mean_inner += [np.mean(mps_sqp)]
            mps_sqp_error_inner += [np.sqrt(np.mean(np.square(exact_sqp - mps_sqp)))]
            mps_fidelity_mean_inner += [np.mean(get_fidelity(exact_state, mps_rdm))]

        # sqp mean and fidelity
        exact_sqp_mean += [exact_sqp_mean_inner]

        mps_sqp_mean += [mps_sqp_mean_inner]
        mps_sqp_error += [mps_sqp_error_inner]
        mps_fidelity_mean += [mps_fidelity_mean_inner]

        bernardi_sqp_mean += [bernardi_sqp_mean_inner]
        bernardi_sqp_error += [bernardi_sqp_error_inner]
        bernardi_fidelity_mean += [bernardi_fidelity_mean_inner]

        onerdm_sqp_mean += [onerdm_sqp_mean_inner]
        onerdm_sqp_error += [onerdm_sqp_error_inner]
        onerdm_fidelity_mean += [onerdm_fidelity_mean_inner]


    exact_sqp_mean = np.mean(np.array(exact_sqp_mean, dtype=float), axis=0)

    mps_sqp_mean = np.mean(np.array(mps_sqp_mean, dtype=float), axis=0)
    mps_sqp_error = np.mean(np.array(mps_sqp_error, dtype=float), axis=0)
    mps_fidelity_mean = np.mean(np.array(mps_fidelity_mean, dtype=float), axis=0)

    bernardi_sqp_mean = np.mean(np.array(bernardi_sqp_mean, dtype=float), axis=0)
    bernardi_sqp_error = np.mean(np.array(bernardi_sqp_error, dtype=float), axis=0)
    bernardi_fidelity_mean = np.mean(np.array(bernardi_fidelity_mean, dtype=float), axis=0)

    onerdm_sqp_mean = np.mean(np.array(onerdm_sqp_mean, dtype=float), axis=0)
    onerdm_sqp_error = np.mean(np.array(onerdm_sqp_error, dtype=float), axis=0)
    onerdm_fidelity_mean = np.mean(np.array(onerdm_fidelity_mean, dtype=float), axis=0)

    suffix = "_" + contraction + "_" + str(bond_dim_list[i])
    np.save(f"{folder}/exact_sqp_mean", exact_sqp_mean)
    
    np.save(f"{folder}/mps_sqp_mean{suffix}", mps_sqp_mean)
    np.save(f"{folder}/mps_sqp_error{suffix}", mps_sqp_error)
    np.save(f"{folder}/mps_fidelity_mean{suffix}", mps_fidelity_mean)

    np.save(f"{folder}/bernardi_sqp_mean", bernardi_sqp_mean)
    np.save(f"{folder}/bernardi_sqp_error", bernardi_sqp_error)
    np.save(f"{folder}/bernardi_fidelity_mean", bernardi_fidelity_mean)

    np.save(f"{folder}/onerdm_sqp_mean", onerdm_sqp_mean)
    np.save(f"{folder}/onerdm_sqp_error", onerdm_sqp_error)
    np.save(f"{folder}/onerdm_fidelity_mean", onerdm_fidelity_mean)


 maximum bond dimension =  2 

--------- iteration = 1 ---------


--------- iteration = 2 ---------
--------- iteration = 3 ---------
--------- iteration = 4 ---------
--------- iteration = 5 ---------
--------- iteration = 6 ---------
--------- iteration = 7 ---------
--------- iteration = 8 ---------
--------- iteration = 9 ---------
--------- iteration = 10 ---------
--------- iteration = 11 ---------
--------- iteration = 12 ---------
--------- iteration = 13 ---------
--------- iteration = 14 ---------
--------- iteration = 15 ---------
--------- iteration = 16 ---------
--------- iteration = 17 ---------
--------- iteration = 18 ---------
--------- iteration = 19 ---------
--------- iteration = 20 ---------
--------- iteration = 21 ---------
--------- iteration = 22 ---------
--------- iteration = 23 ---------
--------- iteration = 24 ---------
--------- iteration = 25 ---------
--------- iteration = 26 ---------
--------- iteration = 27 ---------
--------- iteration = 28 ---------
--------- iteration = 29 ---------
--------- iteration = 30 ---